In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import RegexTokenizer,VectorAssembler,Normalizer,StandardScaler,CountVectorizer,IDF \
,StringIndexer, MinMaxScaler
from pyspark.sql.functions import udf,concat,lit
from pyspark.sql.types import IntegerType

import re

In [2]:
# Creating a Spark Session 
spark=SparkSession.builder.master('local').appName('Word Count').getOrCreate()

## Reading the dataset 

In [8]:
# Printing the columns and the number of rows
df=spark.read.json('../data/Train_onetag_small.json')
df.columns,df.count()

(['Body', 'Id', 'Tags', 'Title', 'oneTag'], 100000)

In [4]:
# Understanding the data
df.show(3)

+--------------------+---+--------------------+--------------------+-------+
|                Body| Id|                Tags|               Title| oneTag|
+--------------------+---+--------------------+--------------------+-------+
|<p>I'd like to ch...|  1|php image-process...|How to check if a...|    php|
|<p>In my favorite...|  2|             firefox|How can I prevent...|firefox|
|<p>I am import ma...|  3|r matlab machine-...|R Error Invalid t...|      r|
+--------------------+---+--------------------+--------------------+-------+
only showing top 3 rows



## Tokenization
Tokenization splits strings into separate words. Spark has a [Tokenizer](https://spark.apache.org/docs/latest/ml-features.html#tokenizer) class as well as RegexTokenizer, which allows for more control over the tokenization process.

In [10]:
regexTokenizer=RegexTokenizer(inputCol='Body',outputCol='words',pattern='\\W')
df=regexTokenizer.transform(df)
df.show()

+--------------------+---+--------------------+--------------------+----------------+--------------------+
|                Body| Id|                Tags|               Title|          oneTag|               words|
+--------------------+---+--------------------+--------------------+----------------+--------------------+
|<p>I'd like to ch...|  1|php image-process...|How to check if a...|             php|[p, i, d, like, t...|
|<p>In my favorite...|  2|             firefox|How can I prevent...|         firefox|[p, in, my, favor...|
|<p>I am import ma...|  3|r matlab machine-...|R Error Invalid t...|               r|[p, i, am, import...|
|<p>This is probab...|  4|     c# url encoding|How do I replace ...|              c#|[p, this, is, pro...|
|<pre><code>functi...|  5|php api file-get-...|How to modify who...|             php|[pre, code, funct...|
|<p>I am using a m...|  6|proxy active-dire...|setting proxy in ...|active-directory|[p, i, am, using,...|
|<p>My image is ca...|  7|           

In [14]:
# Counting the number of paragraphs and links in each body tag
num_para=udf(lambda x : len(re.findall("</p>",x)),IntegerType())
num_links=udf(lambda x: len(re.findall("</a>",x)),IntegerType())

In [16]:
df=df.withColumn('NumParas',num_para(df.Body))
df=df.withColumn('NumLinks',num_links(df.Body))
df.select(['Body','words','NumParas','NumLinks']).show()

+--------------------+--------------------+--------+--------+
|                Body|               words|NumParas|NumLinks|
+--------------------+--------------------+--------+--------+
|<p>I'd like to ch...|[p, i, d, like, t...|       2|       0|
|<p>In my favorite...|[p, in, my, favor...|       2|       0|
|<p>I am import ma...|[p, i, am, import...|       4|       0|
|<p>This is probab...|[p, this, is, pro...|       7|       1|
|<pre><code>functi...|[pre, code, funct...|       2|       0|
|<p>I am using a m...|[p, i, am, using,...|       1|       0|
|<p>My image is ca...|[p, my, image, is...|       9|       0|
|<p>I've decided t...|[p, i, ve, decide...|       4|       0|
|<p>Do you know of...|[p, do, you, know...|       4|       0|
|<p>I'm using SQL ...|[p, i, m, using, ...|       3|       0|
|<p>Some commercia...|[p, some, commerc...|       4|       1|
|<p>This may sound...|[p, this, may, so...|       4|       0|
|<p>how can I move...|[p, how, can, i, ...|       1|       0|
|<p>Few 

In [19]:
# Counting the number of words in each body tag

num_words=udf(lambda x : len(x),IntegerType())
df=df.withColumn("BodyLength",num_words(df.words))

In [20]:
df.select(['Body','words','NumParas','NumLinks','BodyLength']).show()

+--------------------+--------------------+--------+--------+----------+
|                Body|               words|NumParas|NumLinks|BodyLength|
+--------------------+--------------------+--------+--------+----------+
|<p>I'd like to ch...|[p, i, d, like, t...|       2|       0|        83|
|<p>In my favorite...|[p, in, my, favor...|       2|       0|        71|
|<p>I am import ma...|[p, i, am, import...|       4|       0|      3161|
|<p>This is probab...|[p, this, is, pro...|       7|       1|       115|
|<pre><code>functi...|[pre, code, funct...|       2|       0|       148|
|<p>I am using a m...|[p, i, am, using,...|       1|       0|        69|
|<p>My image is ca...|[p, my, image, is...|       9|       0|       112|
|<p>I've decided t...|[p, i, ve, decide...|       4|       0|       161|
|<p>Do you know of...|[p, do, you, know...|       4|       0|       102|
|<p>I'm using SQL ...|[p, i, m, using, ...|       3|       0|        67|
|<p>Some commercia...|[p, some, commerc...|       4

## Vector Assembler
Since NumParas, Numlinks and BodyLength can be a important feature set, we can combine all together with the help of VectorAssembler

In [21]:
assembler=VectorAssembler(inputCols=['NumParas','NumLinks','BodyLength'],outputCol='NumFeatures')
df=assembler.transform(df)

In [22]:
df.select(['Body','NumFeatures']).show()

+--------------------+----------------+
|                Body|     NumFeatures|
+--------------------+----------------+
|<p>I'd like to ch...|  [2.0,0.0,83.0]|
|<p>In my favorite...|  [2.0,0.0,71.0]|
|<p>I am import ma...|[4.0,0.0,3161.0]|
|<p>This is probab...| [7.0,1.0,115.0]|
|<pre><code>functi...| [2.0,0.0,148.0]|
|<p>I am using a m...|  [1.0,0.0,69.0]|
|<p>My image is ca...| [9.0,0.0,112.0]|
|<p>I've decided t...| [4.0,0.0,161.0]|
|<p>Do you know of...| [4.0,0.0,102.0]|
|<p>I'm using SQL ...|  [3.0,0.0,67.0]|
|<p>Some commercia...| [4.0,1.0,134.0]|
|<p>This may sound...| [4.0,0.0,144.0]|
|<p>how can I move...|  [1.0,0.0,19.0]|
|<p>Few month ago ...|  [2.0,0.0,42.0]|
|<p>When you hit a...|  [1.0,0.0,37.0]|
|<p>A lot of frame...|[12.0,0.0,385.0]|
|<p>I'm running a ...| [4.0,0.0,373.0]|
|<p>Hello<br>
I'm ...| [3.0,0.0,209.0]|
|<p>Does anyone kn...| [4.0,1.0,116.0]|
|<p>=) I need your...| [8.0,0.0,390.0]|
+--------------------+----------------+
only showing top 20 rows



## Normalizing the Features
It brings the features on the same scale such that sum of all the elements of a row is 1.

In [23]:
scaler=Normalizer(inputCol='NumFeatures',outputCol='ScaledNumFeatures')
df=scaler.transform(df)

In [25]:
df.select(['Body','NumFeatures','ScaledNumFeatures']).head(2)

[Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", NumFeatures=DenseVector([2.0, 0.0, 83.0]), ScaledNumFeatures=DenseVector([0.0241, 0.0, 0.9997])),
 Row(Body='<p>In my favorite editor (vim), I regularly use ctrl-w to execute a certain action. Now, it quite often happens to me that firefox is the active window (on windows) while I still look at vim (thinking vim is the active window) and press ctrl-w which closes firefox. This is not what I want. Is there a way to stop ctrl-w from closing firefox?</p>\n\n<p>Rene</p>\n', NumFeatures=DenseVector([2.0, 0.0, 71.0]), ScaledNumFeatures=DenseVector([0.0282, 0.0, 0.9996]))]

## StandardScaler
Another way to bring values on the same scale is to Standardize the data such the mean is 0 and all the observations are with in one std units.

In [26]:
scaler2=StandardScaler(inputCol='NumFeatures',outputCol='Scaled2NumFeatures')
scalerModel=scaler2.fit(df)
df=scalerModel.transform(df)

In [27]:
df.select(['Body','NumFeatures','ScaledNumFeatures','Scaled2NumFeatures']).head(2)

[Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", NumFeatures=DenseVector([2.0, 0.0, 83.0]), ScaledNumFeatures=DenseVector([0.0241, 0.0, 0.9997]), Scaled2NumFeatures=DenseVector([0.7037, 0.0, 0.4325])),
 Row(Body='<p>In my favorite editor (vim), I regularly use ctrl-w to execute a certain action. Now, it quite often happens to me that firefox is the active window (on windows) while I still look at vim (thinking vim is the active window) and press ctrl-w which closes firefox. This is not what I want. Is there a way to stop ctrl-w from closing firefox?</p>\n\n<p>Rene</p>\n', NumFeatures=DenseVector([2.0, 0.0, 71.0]), Sc

## Count Vectorizer
Finds the term frequencies of the words

In [29]:
# We restrict our total number of words to 1000
cv=CountVectorizer(inputCol='words',outputCol='TF',vocabSize=1000)
cvmodel=cv.fit(df)
df=cvmodel.transform(df)

In [32]:
df.select(['Body','words','TF']).head(2)

[Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'which', 'file', 'type', 'you', 'upload', 'p', 'p', 'is', 'there', 'a', 'way', 'to', 'check', 'if', 'the', 'uploaded', 'file', 'is', 'an', 'image', 'apart', 

In [52]:
# we can check the featuers with the help of cv.vocabulary
cvmodel.vocabulary

['p',
 'the',
 'i',
 'to',
 'code',
 'a',
 'gt',
 'lt',
 'is',
 'and',
 'pre',
 'in',
 'this',
 'of',
 'it',
 'that',
 'for',
 '0',
 '1',
 'have',
 'my',
 'if',
 'on',
 'but',
 'with',
 'can',
 'not',
 'be',
 'as',
 't',
 'li',
 'from',
 '2',
 's',
 'http',
 'an',
 'm',
 'strong',
 'new',
 'how',
 'do',
 'com',
 'so',
 'or',
 'at',
 'using',
 'when',
 'am',
 'like',
 'class',
 'id',
 'there',
 'get',
 'are',
 'name',
 'what',
 'any',
 'file',
 'string',
 'data',
 'all',
 'which',
 'want',
 'would',
 'amp',
 'use',
 'java',
 'function',
 'public',
 'some',
 '3',
 'text',
 'error',
 'android',
 'value',
 'c',
 'x',
 'href',
 'you',
 'one',
 'by',
 'user',
 'me',
 'server',
 'type',
 'here',
 'way',
 'return',
 'int',
 'will',
 'div',
 'need',
 'then',
 'set',
 'e',
 'system',
 'has',
 'problem',
 'out',
 'php',
 'no',
 'just',
 '4',
 'org',
 'know',
 'html',
 'only',
 'where',
 'page',
 'application',
 '5',
 'thanks',
 'var',
 'br',
 'we',
 'd',
 'should',
 'does',
 'add',
 'n',
 'true',

Thus, we see that the word 'p' occurs 4 times in the body and so on. In this case a lot of weightage is given to common words as their frequencies is much higher than the rare words. Hence we can go for IDF which lowers the weightage on very common words.

## IDF

In [53]:
idf=IDF(inputCol='TF',outputCol='TFIDF')
idfmodel=idf.fit(df)
df=idfmodel.transform(df)

In [54]:
df.head(2)

[Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', words=['p', 'i', 'd', 'like', 'to', 'check', 'if', 'an', 'uploaded', 'file', 'is', 'an', 'image', 'file', 'e', 'g', 'png', 'jpg', 'jpeg', 'gif', 'bmp', 'or', 'another', 'file', 'the', 'problem', 'is', 'that', 'i', 'm', 'using', 'uploadify', 'to', 'upload', 'the', 'files', 'which', 'changes', 'the', 'mime', 'type', 'and', 'gives', 'a', 'text', 'octal', 'or', 'something', 'as', 'the', 'mime', 'type', 'no', 'matter', 'whic

## Handling Categorical Data

In [66]:
df.select('oneTag').show()

+----------------+
|          oneTag|
+----------------+
|             php|
|         firefox|
|               r|
|              c#|
|             php|
|active-directory|
|           other|
|              c#|
|      javascript|
|             sql|
|            .net|
|       algorithm|
|           other|
|   documentation|
|       windows-7|
|             php|
|               r|
|             wpf|
|      javascript|
|             php|
+----------------+
only showing top 20 rows



### StringIndexer

In [69]:
indexer=StringIndexer(inputCol='oneTag',outputCol='Label')
df=indexer.fit(df).transform(df)

In [70]:
df.select('oneTag','Label').show()

+----------------+-----+
|          oneTag|Label|
+----------------+-----+
|             php|  3.0|
|         firefox| 91.0|
|               r| 29.0|
|              c#|  1.0|
|             php|  3.0|
|active-directory|149.0|
|           other|  0.0|
|              c#|  1.0|
|      javascript|  4.0|
|             sql| 14.0|
|            .net| 19.0|
|       algorithm| 46.0|
|           other|  0.0|
|   documentation|292.0|
|       windows-7| 21.0|
|             php|  3.0|
|               r| 29.0|
|             wpf| 23.0|
|      javascript|  4.0|
|             php|  3.0|
+----------------+-----+
only showing top 20 rows



Hence, we can see that php = 3.0 , javascript = 4.0 etc....

# Quick questions

Q1. Select the question with Id = 1112. How many words does its body contain (check the BodyLength column)?

In [73]:
df.select('Body','BodyLength').where("Id=1112").show()

+--------------------+----------+
|                Body|BodyLength|
+--------------------+----------+
|<p>I submitted my...|        63|
+--------------------+----------+



Q2. Create a new column that concatenates the question title and body. Apply the same functions we used before to compute the number of words in this combined column. What's the value in this new column for Id = 5123?

In [74]:
df.columns

['Body',
 'Id',
 'Tags',
 'Title',
 'oneTag',
 'words',
 'NumParas',
 'NumLinks',
 'BodyLength',
 'NumFeatures',
 'ScaledNumFeatures',
 'Scaled2NumFeatures',
 'TF',
 'TFIDF',
 'Label']

In [76]:
# Concatenating 2 columns in PySpark
df=df.withColumn("Combined",concat(df.Title,lit(" "),df.Body))

In [78]:
df.select('Body','Title','Combined','words').head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Title='How to check if an uploaded file is an image without mime type?', Combined="How to check if an uploaded file is an image without mime type? <p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", words=['p', 'i'

In [91]:
df=df.drop('words_combined')

In [92]:
# Checking the number of words and the length in each 
regexTokenizer=RegexTokenizer(inputCol='Combined',outputCol='words_combined',pattern="\\W")
df=regexTokenizer.transform(df)
df.select('Body','Title','Combined','words_combined').head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Title='How to check if an uploaded file is an image without mime type?', Combined="How to check if an uploaded file is an image without mime type? <p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", words_combined=

In [93]:
# counting the bodylength
df=df.withColumn('Combined_bl',num_words(df.words_combined))
df.select('Combined_bl').head()

Row(Combined_bl=96)

In [94]:
# checking value of Id 5123
df.select('Combined_bl').where("Id=5123").show()

+-----------+
|Combined_bl|
+-----------+
|        135|
+-----------+



### Create a vector from the combined Title + Body length column. In the next few questions, you'll try different normalizer/scaler methods on this new column.

In [99]:
df=df.withColumn('NumParas',num_para(df.Combined))
df=df.withColumn('NumLinks',num_links(df.Combined))
df.select(['Body','words','NumParas','NumLinks']).show()

+--------------------+--------------------+--------+--------+
|                Body|               words|NumParas|NumLinks|
+--------------------+--------------------+--------+--------+
|<p>I'd like to ch...|[p, i, d, like, t...|       2|       0|
|<p>In my favorite...|[p, in, my, favor...|       2|       0|
|<p>I am import ma...|[p, i, am, import...|       4|       0|
|<p>This is probab...|[p, this, is, pro...|       7|       1|
|<pre><code>functi...|[pre, code, funct...|       2|       0|
|<p>I am using a m...|[p, i, am, using,...|       1|       0|
|<p>My image is ca...|[p, my, image, is...|       9|       0|
|<p>I've decided t...|[p, i, ve, decide...|       4|       0|
|<p>Do you know of...|[p, do, you, know...|       4|       0|
|<p>I'm using SQL ...|[p, i, m, using, ...|       3|       0|
|<p>Some commercia...|[p, some, commerc...|       4|       1|
|<p>This may sound...|[p, this, may, so...|       4|       0|
|<p>how can I move...|[p, how, can, i, ...|       1|       0|
|<p>Few 

In [108]:
df=df.drop('CombinedFeature','CFs2')

In [109]:
assembler=VectorAssembler(inputCols=['Combined_bl'],outputCol='CombinedFeature')
df=assembler.transform(df)
df.select('Combined_bl','CombinedFeature').head(2)

[Row(Combined_bl=96, CombinedFeature=DenseVector([96.0])),
 Row(Combined_bl=83, CombinedFeature=DenseVector([83.0]))]

Q3. Using the StandardScaler method (scaling both the mean and the standard deviation) what's the normalized value for question Id = 512?

In [110]:
scaler2=StandardScaler(inputCol='CombinedFeature',outputCol='CFs2',withStd=True,withMean=True)
df=scaler2.fit(df).transform(df)

In [112]:
df.select('CFs2').where("Id=512").head()

Row(CFs2=DenseVector([-0.6417]))

Q4. Using the MinMAxScaler method what's the normalized value for question Id = 512?


In [124]:
max_val=df.agg({'CombinedFeature':'max'}).collect()[0]['max(CombinedFeature)']
min_val=df.agg({'CombinedFeature':'min'}).collect()[0]['min(CombinedFeature)']

In [125]:
min_val[0]

10.0

In [130]:
scaler3=MinMaxScaler(inputCol='CFs2',outputCol='CFs3')
df=scaler3.fit(df).transform(df)

In [131]:
df.select('CFs3').where("Id=512").head()

Row(CFs3=DenseVector([0.0062]))